# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [24]:
from copy import deepcopy
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [121]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """

    def __init__(self, location, car_id, gap=28, speed_limit=33, start_speed=27):
        self.location = location
        self.gap = gap
        self.desired_speed = speed_limit
        self.speed = start_speed
        self.size = 5
        self.bumper = 0
        self.update_bumper()
        self.car_id = car_id

    def __str__(self):
        return 'Car(location={},car_id={},bumper={},gap={},speed={})'.format(
                self.location, self.car_id, self.bumper, self.gap, self.speed)

    def __repr__(self):
        return self.__str__()

    def drive(self, car_ahead):

        if self.location + self.speed >= car_ahead.bumper:
            self.speed = 0
            self.location = car_ahead.bumper - 1
        elif self.location + self.speed > car_ahead.bumper - self.speed:
            self.speed = car_ahead.speed
        elif random.random() < 0.1:
            self.speed -= 2
            if self.speed < 0:
                self.speed = 0
        elif self.speed < self.desired_speed:
            self.speed += 2
            if self.speed > self.desired_speed:
                self.speed = self.desired_speed

        self.location += self.speed
        self.bumper = self.location - self.size + 1
        if self.location >= 1000:
            print("I'm off the road, man!")
            return 'off_the_road'
        else:
            return 'on_the_road'

    def update_bumper(self):
        self.bumper = self.location - self.size + 1


In [122]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - (1000 - sum(vehicle.size)) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self):
        self.total_vehicle_space = (30 * 5)
        self.initial_gap = int((1000 - self.total_vehicle_space) / 30)
        self.vehicles = [Car((4 + int(33.333333333*n)),n, self.initial_gap) for n in range(30)]
        self.vehicles[-1].gap = (1000 - self.vehicles[-1].location)

In [123]:
class HighwaySim:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
    - Report stats; return traffic jam status
    - Pop, Append

    Collaborators:
    - Car
    - Road
    """

    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.is_traffic = []

    def run_sim(self, duration=1):
        while self.ticks < duration:
            self.iterate()
            self.ticks += 1
        if self.is_traffic.count(True) > 0:
            return True
        else:
            return False

    def iterate(self):
        off_the_road = []
        num_cars = len(self.road.vehicles)
        for idx in range(num_cars):
            v = self.road.vehicles[- idx - 1]
            if idx > 0:
                car_ahead = deepcopy(self.road.vehicles[- idx])
            else:
                car_ahead = deepcopy(self.road.vehicles[0])
                car_ahead.location += 1000
                if car_ahead.gap < car_ahead.speed:
                    car_ahead.location += car_ahead.gap
                else:
                    car_ahead.location += car_ahead.speed
                car_ahead.bumper = car_ahead.location - car_ahead.size + 1

            if v.drive(car_ahead) == 'off the road':
                off_the_road.append(- idx - 1)
            
            if car_ahead.bumper - v.location > 0:
                v.gap = car_ahead.bumper - v.location
            else:
                v.gap = car_ahead.bumper + 1000 - v.location
                
            v.update_bumper()

            if v.speed == 0:
                self.is_traffic.append(True)
            else:
                self.is_traffic.append(False)

        while len(off_the_road) > 0:
            print(self.road.vehicles[-1])
            off_car = self.road.vehicles.pop(-1)
            off_car.location -= 1000
            if off_car.location >= off_car.size - 1:
                off_car.update_bumper()
            else:
                off_car.bumper = off_car.location - off_car.size + 1 + 1000
            self.road.vehicles.insert(0, off_car)
            off_the_road.pop(0)
        print(self.road.vehicles[-1])

In [126]:
sim1 = HighwaySim()
sim1.run_sim(2)
sim1.road.vehicles

Car(location=999,car_id=29,bumper=995,gap=28,speed=29)
I'm off the road, man!
Car(location=1030,car_id=29,bumper=1026,gap=28,speed=31)


[Car(location=60,car_id=0,bumper=56,gap=29,speed=27),
 Car(location=93,car_id=1,bumper=89,gap=29,speed=27),
 Car(location=126,car_id=2,bumper=122,gap=29,speed=27),
 Car(location=159,car_id=3,bumper=155,gap=30,speed=27),
 Car(location=193,car_id=4,bumper=189,gap=29,speed=27),
 Car(location=226,car_id=5,bumper=222,gap=29,speed=27),
 Car(location=259,car_id=6,bumper=255,gap=30,speed=27),
 Car(location=293,car_id=7,bumper=289,gap=29,speed=27),
 Car(location=326,car_id=8,bumper=322,gap=29,speed=27),
 Car(location=359,car_id=9,bumper=355,gap=30,speed=27),
 Car(location=393,car_id=10,bumper=389,gap=29,speed=27),
 Car(location=426,car_id=11,bumper=422,gap=29,speed=27),
 Car(location=459,car_id=12,bumper=455,gap=30,speed=27),
 Car(location=493,car_id=13,bumper=489,gap=29,speed=27),
 Car(location=526,car_id=14,bumper=522,gap=29,speed=27),
 Car(location=559,car_id=15,bumper=555,gap=30,speed=27),
 Car(location=593,car_id=16,bumper=589,gap=25,speed=27),
 Car(location=622,car_id=17,bumper=618,gap=33

In [6]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        trial_data = HighwaySim()
        trial_stats.append(trial_data)
    return trial_stats

In [7]:
highway_1000_data = highway_trials()

In [8]:
print(highway_1000_data.count(True))

0
